In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from pyvis import network as net
import json
from os import listdir

import datashader as ds
import datashader.transfer_functions as tf
from datashader.bundling import hammer_bundle

In [ ]:
pd.options.display.max_columns = None

Load the JSON files 

In [ ]:
def load_data(directory):
    # Load json files 
    json_data_asctb = {}
    for filename in listdir(directory):
        if '.json' in filename:
            #print(filename)
            with open(f'{directory}{filename}', encoding='utf8') as json_file:
                data = json.load(json_file)

            json_data_asctb[filename[:-5]] = data
        #print(json_data_asctb)
    return json_data_asctb

In [ ]:
json_data_asctb=load_data('v14/v14/') ## Download the JSON files from ASCT+B Reporter

In [ ]:
json_data_asctb #view the data

Order of the organs

In [ ]:
organ_order = ['trachea','main_bronchus','lung', 'heart', 'spinal_cord', 'brain', 'eye', 'skin',
               'thymus', 'lymph_nodes', 'spleen', 'liver', 'pancreas', 
               'small_intestine', 'large_intestine', 'kidney', 'urinary_bladder', 'ureter', 'prostate', 
               'ovary', 'fallopian_tube', 'uterus', 'placenta_full_term', 'knee', 'bone_marrow']

In [ ]:
print('New organs:')
[organ for organ in json_data_asctb.keys() if organ not in organ_order+['blood', 'blood_vasculature', 'lymph_vasculature', 'peripheral_nervous_system']]

In [ ]:
json_data_asctb = {organ: json_data_asctb[organ] for organ in organ_order}

In [ ]:
json_data_asctb.keys()

In [ ]:
def combine_data(json_dat, remove_ctct=True, combine=True, blood=False, verbose=False):
    """
    combine_data takes the json data of the organs and returns the node and edge lists in pandas dataframes

    :param json_dat: data of all the organs in a json format that was loaded by the load_data function
    :param remove_ctct: There should not be edges between CT and CT, however sometimes there can be mistakes in the data files. 
    :param combine: if true then the organs are combined into one network, and the function outputs 2 dataframes: nodes and edges. 
                    if false, then the organs are not combined, the function returns a node and an edge table for each organ.
    :param blood: By default it is false, and in this case, the function ignores the following tables: 'blood_pelvis', 'blood_vasculature', 
                 'lymph_vasculature', and 'peripheral_nervous_system'. If true, then 'blood_vasculature' is not excluded.

    """ 
    nodes_data = []
    edges_data = []
    organ_names = ['body']
    
    # The numbering of the nodes starts from 1 in each organ
    # If we combine the organs, we need a new id
    node_id_start_point = 1 
    
    #Add the 'body' node to the nodes data:
    body_df = pd.DataFrame(json_dat['brain']['nodes']).head(1)
    body_df.index = body_df['id']
    body_df['id_old'] = 0
    body_df['organ'] = 'body'
    body_df['ontology_id'] = 'UBERON:0013702'
    
    nodes_data += [body_df[['id', 'id_old', 'name', 'type', 'organ', 'ontology_id']]]
    
    if not combine:
        body_data = body_df[['id', 'id_old', 'name', 'type', 'organ', 'ontology_id']].copy()
    
    if blood:
        exclude = ['blood_pelvis', 'lymph_vasculature', 'peripheral_nervous_system']
    else:
        exclude = ['blood_pelvis', 'blood_vasculature', 'lymph_vasculature', 'peripheral_nervous_system']
    
    for name in json_dat.keys():
        if name in exclude:
            continue
        else:
            # print(name)
            organ_names += [name]
            # Combine nodes:
            organ_nodes = pd.DataFrame(json_dat[name]['nodes']).iloc[1:,:]
            not_as_ct_nodes = organ_nodes[~organ_nodes['type'].isin(['AS', 'CT'])]['id'] #filter for AS and CT
            organ_nodes['id_old'] = organ_nodes['id'].copy()
            if combine:
                organ_nodes['id'] = organ_nodes['id'].add(node_id_start_point-1)
            organ_nodes.index = organ_nodes['id']
            # organ_nodes = organ_nodes.drop(columns=['id'])
            organ_nodes = organ_nodes[organ_nodes['type'].isin(['AS', 'CT'])]
            organ_nodes['organ'] = name
            organ_nodes['ontology_id'] = organ_nodes['metadata'].apply(lambda x: x['ontologyId'])
            organ_nodes = organ_nodes[['id', 'id_old', 'name', 'type', 'organ', 'ontology_id']]
            

            # Combine edges:
            organ_edges = pd.DataFrame(json_dat[name]['edges'])
            ## Remove CT-CT edges:
            if remove_ctct:
                organ_nodes_original = pd.DataFrame(json_dat[name]['nodes'])
                organ_nodes_original.index = organ_nodes_original['id']
                organ_edges['source_type'] = organ_edges['source'].apply(lambda x: organ_nodes_original.loc[x]['type'])
                organ_edges['target_type'] = organ_edges['target'].apply(lambda x: organ_nodes_original.loc[x]['type'])
                organ_edges = organ_edges[(organ_edges['source_type']=='AS')|(organ_edges['target_type']=='AS')]
            ##rename edges due to colliding ids
            organ_network = nx.from_pandas_edgelist(organ_edges)
            #remove unnecessary edges:
            organ_network.remove_nodes_from(not_as_ct_nodes.to_list())
            # Remove body-organ edge (we will add it later)
            organ_network.remove_edge(0, 1)
            organ_network.remove_node(0)
            #rename nodes
            # organ_network = nx.relabel_nodes(organ_network, organ_nodes['name'].to_dict())
            if combine:
                organ_network = nx.convert_node_labels_to_integers(organ_network, first_label=node_id_start_point)
            #add body-organ edge
            # organ_network.add_node(0) 
            # nx.set_node_attributes(organ_network, {0: {"type": 'AS', "name": "Body", 'organ': 'body', 'ontology_id': 'UBERON:0013702'}})
            if combine:
                organ_network.add_edge(0, node_id_start_point) #add body-organ edge
                #v1.4 correction:
                if name=='lung':
                    organ_network.add_edge(node_id_start_point, 63+node_id_start_point-1)  #connect lung to pulmonary vascular system
                    organ_network.add_edge(node_id_start_point, 69+node_id_start_point-1)  #connect lung to systemic artery
                    organ_network.add_edge(node_id_start_point, 73+node_id_start_point-1)  #connect lung to venous blood vessel

                elif name=='trachea':
                    organ_network.add_edge(node_id_start_point, 15+node_id_start_point-1)  #connect trachea to bronchial vein
                    organ_network.add_edge(node_id_start_point, 18+node_id_start_point-1)  #connect trachea to bronchial artery
                    
                elif name=='main_bronchus':
                    organ_network.add_edge(node_id_start_point, 10+node_id_start_point-1) #connect trachea to bronchial artery
                    organ_network.add_edge(node_id_start_point, 13+node_id_start_point-1) #connect trachea to bronchial vein
                    
            else:
                organ_network.add_edge(0, 1) #add body-organ edge
                
                #v1.4 corrections:
                if name=="lung":
                    organ_network.add_edge(0, 63) #connect lung to pulmonary vascular system
                    organ_network.add_edge(0, 69) #connect lung to systemic artery
                    organ_network.add_edge(0, 73) #connect lung to venous blood vessel

                elif name=="trachea":
                    organ_network.add_edge(0, 15) #connect trachea to bronchial vein
                    organ_network.add_edge(0, 18) #connect trachea to bronchial artery

                elif name=='main_bronchus':
                    organ_network.add_edge(0, 10) #connect trachea to bronchial artery
                    organ_network.add_edge(0, 13) #connect trachea to bronchial vein


                

            # Create CT duplicates
            organ_CT_nodes = organ_nodes[organ_nodes['type']=='CT'].copy()
            organ_CT_nodes['degree'] = dict(organ_network.degree(organ_CT_nodes['id'])).values()
            
            ct_nodes_to_copy = organ_CT_nodes[organ_CT_nodes['degree']>1]['id'].values
            edges_to_remove = []
            edges_to_add = []
            
            for ct_node in ct_nodes_to_copy:
                as_neighbors_of_ct = list(nx.neighbors(organ_network, ct_node))
                for ind, as_neighbor in enumerate(as_neighbors_of_ct):
                    if ind==0:
                        continue
                    else:
                        ct_node_details = organ_nodes.loc[ct_node].to_dict()
                        new_node_id = organ_nodes['id'].max()+1
                        ct_node_details['id'] = new_node_id
                        # organ_nodes = organ_nodes.append(ct_node_details, ignore_index=False)
                        organ_nodes.loc[new_node_id] = ct_node_details
                        
                        edges_to_remove += [(as_neighbor, ct_node)]
                        edges_to_add += [(as_neighbor, new_node_id)]

            if verbose:
                print(name)
                print(f'Number of deleted edges: {len(edges_to_remove)}')
                print(f'Number of added edges: {len(edges_to_add)}')
                
            organ_network.remove_edges_from(edges_to_remove)
            organ_network.add_edges_from(edges_to_add)               
            
            print(nodes_data)
            nodes_data += [organ_nodes]
            if not combine:
                nodes_data[-1] = pd.concat([body_data, nodes_data[-1]])
            
            node_id_start_point = max(organ_network.nodes)+1
            
            organ_edges = nx.to_pandas_edgelist(organ_network)[['source', 'target']]
            print(f"{name} is tree: {nx.is_tree(organ_network)}", end='\n_____________\n')
            edges_data += [organ_edges]

    if combine:
        return pd.concat(nodes_data, ignore_index=True), pd.concat(edges_data, ignore_index=True)
    else:
        return dict(zip(organ_names, nodes_data)), dict(zip(organ_names[1:], edges_data))

Combine the nodes and edges of the organs. (The code checks whether the graph of the organs are trees)

In [ ]:
nodes, edges = combine_data(json_data_asctb, remove_ctct=True, combine=True)

In [ ]:
nodes['organ'].unique()

In [ ]:
edges.head()

In [ ]:
FTUs = ['UBERON:0001229', 'UBERON:0001285', 'UBERON:0004205', 'UBERON:0004204', 'UBERON:0004203', 'UBERON:0004193', 'UBERON:0001289', 'UBERON:0001291', 'UBERON:0013485', 'UBERON:0004647', 'UBERON:0008870', 'UBERON:8410043', 'UBERON:0000006', 'UBERON:0001263', 'UBERON:0007329', 'UBERON:0004179', 'UBERON:0001003', 'UBERON:0002067', 'UBERON:0001213', 'UBERON:0001959', 'UBERON:0001250', 'UBERON:0002125']

In [ ]:
nodes[nodes['ontology_id'].isin(FTUs)]['ontology_id'].unique()

## Construction of network 

In [ ]:
whole_graph = nx.from_pandas_edgelist(edges)

In [ ]:
def get_parent(node, graph):
    neighbors = nx.neighbors(graph, node)
    # print(len([x for x in neighbors if x<node]))
    return min(nx.neighbors(graph, node))

In [ ]:
nodes['parent']= nodes['id'].apply(get_parent, graph=whole_graph)

Size of network in v1.4

In [ ]:
whole_graph.number_of_nodes()

In [ ]:
whole_graph.number_of_edges()

In [ ]:
nodes['color'] = nodes.apply(lambda row: '#56a04e' if row['ontology_id'] in FTUs else ('#984ea0' if row['type']=='AS' else '#ff7f00'), axis=1)

In [ ]:
nodes_json = nodes[['id', 'name', 'parent', 'type', 'ontology_id', 'color']].to_dict(orient='index')

In [ ]:
del(nodes_json[0]['parent'])

In [ ]:
nodes_json = [nodes_json[i] for i in nodes_json.keys()]

In [ ]:
with open('human_atlas_v1.4', 'w', encoding='utf8') as f:
    f.write('[\n')
    for item in nodes_json:
        f.write(f"{json.dumps(item)},\n".replace("'", ''))
    f.write(']')

In [ ]:
nodes['type'].value_counts()

### Female network 

In [ ]:
def construct_network(jason_data, filename):
    nodes_df, edges_df = combine_data(jason_data, remove_ctct=True, combine=True)
    graph = nx.from_pandas_edgelist(edges_df)
    nodes_df['parent']= nodes_df['id'].apply(get_parent, graph=graph)
    nodes_df['color'] = nodes_df.apply(lambda row: '#56a04e' if row['ontology_id'] in FTUs else ('#984ea0' if row['type']=='AS' else '#ff7f00'), axis=1)
    
    nodes_df['organ_label'] = nodes_df.apply(lambda row: row['name'] if row['id_old']==1 else '', axis=1)
    nodes_json = nodes_df[['id', 'name', 'parent', 'type', 'ontology_id', 'color', 'organ', 'organ_label']].to_dict(orient='index')
    del(nodes_json[0]['parent'])
    nodes_json = [nodes_json[i] for i in nodes_json.keys()]
    
    with open(f'{filename}.json', 'w', encoding='utf8') as f:
        f.write('[\n')
        for item in nodes_json:
            f.write(f"{json.dumps(item)},\n".replace("'", ''))
        f.write(']')
    
    return nodes_df, edges_df

In [ ]:
fem_json_data = json_data_asctb.copy()

In [ ]:
del(fem_json_data['prostate'])

In [ ]:
fem_nodes, fem_edges = construct_network(fem_json_data, 'v14/v14/')

In [ ]:
fem_edges

In [ ]:
fem_nodes[fem_nodes['organ']=='prostate']

In [ ]:
fem_nodes.color.value_counts()

In [ ]:
fem_nodes[fem_nodes['color']=='#56a04e'].sort_values('ontology_id') #.to_excel('ftus.xlsx')

### Male network 

In [ ]:
male_json_data = json_data_asctb.copy()

In [ ]:
del(male_json_data['fallopian_tube'])

In [ ]:
del(male_json_data['ovary'])

In [ ]:
del(male_json_data['uterus'])

In [ ]:
del(male_json_data['placenta_full_term'])

In [ ]:
male_nodes, male_edges = construct_network(male_json_data, 'v14/v14/')

In [ ]:
male_nodes

In [ ]:
male_edges

In [ ]:
male_nodes[male_nodes['name'].apply(lambda x: 'prostate' in str(x).lower())]

# Vega radial tree 

* Config saved in (fe)male_wing_viz.json. 
* Copy human_atlas_(fe)_male_v13 in the values 
* Export SVG and JSON. 

In [ ]:
def construct_network_create_vega_viz(jason_data, filename):
    nodes_df, edges_df = combine_data(jason_data, remove_ctct=True, combine=True)
    graph = nx.from_pandas_edgelist(edges_df)
    nodes_df['parent']= nodes_df['id'].apply(get_parent, graph=graph)
    nodes_df['color'] = nodes_df.apply(lambda row: '#56a04e' if row['ontology_id'] in FTUs else ('#984ea0' if row['type']=='AS' else '#ff7f00'), axis=1)
    
    nodes_df['organ_label'] = nodes_df.apply(lambda row: row['name'] if row['id_old']==1 else '', axis=1)
    nodes_json = nodes_df[['id', 'name', 'parent', 'type', 'ontology_id', 'color', 'organ', 'organ_label']].to_dict(orient='index')
    del(nodes_json[0]['parent'])
    nodes_json = [nodes_json[i] for i in nodes_json.keys()]
    

    with open('v14/vega_config.json', encoding='utf8') as json_file:
        config = json.load(json_file)

    config['data'][0]['values'] = nodes_json
    
    
    # Writing to sample.json
    with open(f"v14/v14_{filename}.json", "w") as outfile:
        outfile.write(json.dumps(config, indent=4))
    
    return nodes_df, edges_df

In [ ]:
_, _ = construct_network_create_vega_viz(male_json_data, 'male')

In [ ]:
_, _ = construct_network_create_vega_viz(fem_json_data, 'female')

# Vasculature 

In [ ]:
asctb_blood_vasc_nodes = pd.DataFrame(load_data('./asct+b/v14/')['blood_vasculature']['nodes'])

In [ ]:
asctb_blood_vasc_nodes['ontologyId'] = asctb_blood_vasc_nodes['metadata'].apply(lambda x: x['ontologyId'])
asctb_blood_vasc_nodes['label'] = asctb_blood_vasc_nodes['metadata'].apply(lambda x: x['label'])
asctb_blood_vasc_nodes.drop(columns=['metadata'], inplace=True)

In [ ]:
asctb_blood_vasc_nodes.head()

In [ ]:
asctb_blood_vasc_nodes['type'].value_counts()

In [ ]:
asctb_blood_vasc_edges = pd.DataFrame(load_data('./asct+b/v14/')['blood_vasculature']['edges'])

In [ ]:
asctb_blood_vasc_edges

In [ ]:
vasc_data = pd.read_csv('./Vessel.csv', encoding="ISO-8859-1")[['BranchesFrom', 'Vessel', 'ASID', 'VesselType', 'BodyPart', 'BodyPartID', 'PathFromHeart', 'PathFromHeartWithIDs']]

In [ ]:
vasc_data.isna().sum().sort_values(ascending=False)

In [ ]:
vasc_data.head()

In [ ]:
len(vasc_data)

In [ ]:
# vasc_data = pd.read_excel('./Blood_Vasculature_v1.2_EXTENDED.xlsx', sheet_name='BodyPartVessel')
# vasc_data = pd.read_excel('./Blood_Vasculature_v1.3_EXTENDED.xlsx', sheet_name='BodyPartVessel')
vasc_data

In [ ]:
# beta_vasc = pd.read_excel('BetaVasculatureMappingtoASCT+BTablesforMarcell08.25.22.xlsx')

In [ ]:
# beta_vasc

In [ ]:
# beta_vasc[beta_vasc.duplicated(subset=['Vessel'], keep=False)]

Organs in the ASCT+B tables:

In [ ]:
nodes['organ'].unique()

Organs/body parts in the vasculature data:

In [ ]:
vasc_data['BodyPart'].unique()

Matching body parts based on uberon id (BodyPartID)

In [ ]:
nodes[nodes.ontology_id.isin(set(vasc_data['BodyPartID']))]

In [ ]:
# nodes[nodes.ontology_id.isin(set(beta_vasc['ASCT+B Mapped Uberon ID']))]

In [ ]:
vasc_data[vasc_data.BodyPartID.isin(set(nodes['ontology_id']))].BodyPart.unique()

In [ ]:
# beta_vasc[beta_vasc['ASCT+B Mapped Uberon ID'].isin(set(nodes['ontology_id']))]['ASCT_B Mapped Table'].unique()

The FMA IDs have a different format in the vascular data set 

In [ ]:
def transform_id(input_id):
    return str(input_id).lower().replace(':', '')

The set of uberon/fma IDs that are in the ASCT+B tables:

In [ ]:
ids_in_organtables = set([transform_id(node) for node in set(nodes.ontology_id)])

Get the mathching and non-matching vessels:

In [ ]:
vasc_data['is_in_organ_table'] = vasc_data['ASID'].apply(lambda x: 1 if transform_id(x) in ids_in_organtables else 0)
vasc_data['not_in_organ_table'] = vasc_data['ASID'].apply(lambda x: 0 if transform_id(x) in ids_in_organtables else 1)
vasc_data['#vessels'] = 1

In [ ]:
vasc_data[vasc_data['is_in_organ_table']==1]

In [ ]:
vasc_data['is_in_organ_table'].sum()

## Circles/bubbles

In [ ]:
data_for_circles = (vasc_data[vasc_data.BodyPartID.isin(set(nodes['ontology_id']))][['BodyPart', 'BodyPartID', 'is_in_organ_table',
                                                                                     'not_in_organ_table','#vessels']]
                    .groupby(['BodyPart', 'BodyPartID']).sum()
                    # .drop(index=[('testis', 'UBERON:0000473'), 
                    #              ('thyroid gland','UBERON:0002046'),
                    #              ('heart chamber', 'UBERON:0004151')])
                    .rename(columns={'is_in_organ_table': '#matching vessels', 'not_in_organ_table': '#vessels not in ASCT+B'})
                   )
                    
data_for_circles.reset_index()

In [ ]:
rest = (vasc_data[~vasc_data.BodyPartID.isin(set(vasc_data['BodyPartID']).intersection(set(nodes['ontology_id'])))][['BodyPart', 'BodyPartID', 
                                                                                                                      'is_in_organ_table', 'not_in_organ_table', '#vessels']]
                    .groupby(['BodyPart', 'BodyPartID']).sum()
                    # .drop(index=[('testis', 'UBERON:0000473'), 
                    #              ('thyroid gland','UBERON:0002046'),
                    #              ('heart chamber', 'UBERON:0004151')])
                    .rename(columns={'is_in_organ_table': '#matching vessels', 'not_in_organ_table': '#vessels not in ASCT+B'})
                   )
                    
rest

### The four matching vessels from heart chamber are added to heart manually. The one vessel in pelvis is added to placenta. The one matching vessel in testis is matching with a vessel in the ovary. 
( There used to be 2 thyroid gland matching vessels that needed to be added manually but in the v1.3 version it is not needed anymore. There used to be 2 thyroid gland matching vessels that needed to be added manually but version>=1.3 it is not needed anymore.)

### The query of these vessels can be seen below:

Vessels in the heart chamber:

In [ ]:
vasc_data[vasc_data.BodyPartID=="UBERON:0004151"]

heart chamber in ASCT+B:

In [ ]:
nodes[nodes.ontology_id.isin(['UBERON:0002079', 'UBERON:0002084', 'UBERON:0002078', 'UBERON:0002080'])]

Vessels in pelvis

In [ ]:
vasc_data[vasc_data.BodyPartID=="UBERON:0002355"].sort_values('is_in_organ_table')

Only 'umbilical artery' is in ASCT+B, and they are part of the placenta organ:

In [ ]:
nodes[nodes.ontology_id=='UBERON:0001310']

In [ ]:
nodes[nodes.name=='Placenta']

Vessels in testis:

In [ ]:
vasc_data[vasc_data.BodyPartID=="UBERON:0000473"]

Only 'pampiniform venous plexus' is matching, but this vessel is in ovary and not in testis:

In [ ]:
nodes[nodes.ontology_id=='UBERON:0001330']

Vessels in ovary

In [ ]:
vasc_data[vasc_data.BodyPartID=="UBERON:0000992"]

There's only one vessel in ovary that is also in ASCT+B:

In [ ]:
nodes[nodes.ontology_id=='UBERON:0001190']

After manually performed merge of brain & cerbellum, heart & heart chamber, and thyroid & lung, large intestine & rectum we get the following table:

In [ ]:
circles = data_for_circles.reset_index(level=1).copy()

rest_circles = rest.reset_index(level=1).copy()

#brain + cerebellum
circles.at['brain', '#matching vessels'] += circles.at['cerebellum', '#matching vessels']
circles.at['brain', '#vessels not in ASCT+B'] += circles.at['cerebellum', '#vessels not in ASCT+B'] 
circles.at['brain', '#vessels'] += circles.at['cerebellum', '#vessels'] 

#large intestine + rectum
circles.at['large intestine', '#matching vessels'] += circles.at['rectum', '#matching vessels']
circles.at['large intestine', '#vessels not in ASCT+B'] += circles.at['rectum', '#vessels not in ASCT+B'] 
circles.at['large intestine', '#vessels'] += circles.at['rectum', '#vessels'] 

#heart + heart chamber
circles.at['heart', '#matching vessels'] += rest_circles.at['heart chamber', '#matching vessels']
circles.at['heart', '#vessels not in ASCT+B'] += rest_circles.at['heart chamber', '#vessels not in ASCT+B'] 
circles.at['heart', '#vessels'] += rest_circles.at['heart chamber', '#vessels']

# ovary +1
circles.at['ovary', '#matching vessels'] += 1
circles.at['ovary', '#vessels'] += 1

# placenta
circles.loc['placenta'] = ['UBERON:0001987', 1, 0, 1] 

circles = circles.drop(index=['cerebellum', 'rectum'])

In [ ]:
circles

In [ ]:
match_vess = circles.to_dict()['#matching vessels']

In [ ]:
nomatch_vess = circles.to_dict()['#vessels not in ASCT+B']

In [ ]:
nomatch_vess

## Separate bubbles 

Generation of the bubbles:

In [ ]:
# for organ in match_vess.keys():
#     plt.figure(frameon=False, figsize=(10,10))
#     plt.axes().set_aspect('equal')
#     plt.axis('off')
#     plt.xlim(-0.025,0.025)
#     plt.ylim(-0.025,0.025)
#     size_scale = 500
#     if match_vess[organ]<=nomatch_vess[organ]:
#         big_d = nomatch_vess[organ]+.000001
#         small_d = match_vess[organ]+.000001
#         big_color='tab:red'
#         small_color='black'
#     else:
#         big_d=match_vess[organ]
#         small_d = nomatch_vess[organ]
#         big_color='black'
#         small_color='tab:red'
#     plt.scatter([0], [0], s=big_d*size_scale, c=big_color, alpha=0.4)
#     plt.scatter([0], [(-np.sqrt(big_d)+np.sqrt(small_d))/990], s=small_d*size_scale, c=small_color, alpha=0.4)
#     plt.savefig(f'{griffin_organ_order.index(str(organ).replace(" ", "_"))}_{organ}.svg', bbox_inches='tight', pad_inches=0)


#     # show the graph
#     plt.show()

### Rotation angle of the bubbles

We need to extract the positions of the nodes of the organs and from the coordinates we compute the angles for the rotations

__Female organs__

In [ ]:
with open('female_wing_viz_v13.json', encoding='utf8') as json_file:
    viz = json.load(json_file)

In [ ]:
female_wing_viz = viz['data'][0]['values']

In [ ]:
body_x = female_wing_viz[0]['x']
body_y = female_wing_viz[0]['y']
fem_rots_for_organs = dict()
coordinates = dict()
for asct in female_wing_viz:
     if asct['name'] in fem_nodes_c[fem_nodes_c['id_old']==1].name.values:
        fem_rots_for_organs[asct['name']] = -round(np.arctan((asct['x']-body_x)/(body_y-asct['y']))*180/np.pi, 3)
        coordinates[asct['name']] = (round(body_x + 14*(asct['x']-body_x),2), round(body_y + 14*(asct['y']-body_y), 2))

In [ ]:
fem_rots_for_organs

In [ ]:
coordinates

In [ ]:
with open('male_wing.json', encoding='utf8') as json_file:
    viz = json.load(json_file)

In [ ]:
male_wing_viz = viz['data'][0]['values']

In [ ]:
body_x = male_wing_viz[0]['x']
body_y = male_wing_viz[0]['y']
male_rots_for_organs = dict()
coordinates = dict()
for asct in male_wing_viz:
    if asct['name'] in male_nodes_c[male_nodes_c['id_old']==1].name.values:
        male_rots_for_organs[asct['name']] = -round(np.arctan((asct['x']-body_x)/(body_y-asct['y']))*180/np.pi, 3)
        coordinates[asct['name']] = (round(body_x + 14*(asct['x']-body_x),2), round(body_y + 14*(asct['y']-body_y), 2))

In [ ]:
male_rots_for_organs

In [ ]:
coordinates

## Two layers of bubbles - no rotation is needed, first black then red 

In [ ]:
organ_order

In [ ]:
with open('viz_v13/male_wing_viz_v13.json', encoding='utf8') as json_file:
    viz = json.load(json_file)

In [ ]:
male_wing_viz = viz['data'][0]['values']

In [ ]:
nomatch_vess

In [ ]:
match_vess

In [ ]:
pd.Series([asct['name'] for asct in male_wing_viz if asct['name'] in male_nodes[male_nodes['id_old']==1].name.values]).unique()

In [ ]:
body_x = male_wing_viz[0]['x']
body_y = male_wing_viz[0]['y']
# print('Body original: ', body_x, body_y)
# center_x = 1720/2
# center_y = 1720/2
# print('Center:', center_x, center_y)
coordinates_black = dict()
coordinates_red = dict()
for asct in male_wing_viz:
    if asct['name'] in male_nodes[male_nodes['id_old']==1].name.values:
        # male_rots_for_organs[asct['name']] = -round(np.arctan((asct['x']-body_x)/(body_y-asct['y']))*180/np.pi, 3)
        coordinates_red[asct['name']] = (round(body_x-10 + 13.3*(asct['x']-body_x),2), round(body_y + 13.3*(asct['y']-body_y), 2))
        coordinates_black[asct['name']] = (round(body_x-10 + 12*(asct['x']-body_x),2), round(body_y + 12*(asct['y']-body_y), 2))

mapping = {'Respiratory System': 'lung', 'heart': 'heart', 'brain': 'brain', 'eye': 'eye', 'skin': 'skin', 'thoracic thymus': 'thymus', 'lymph node': 'lymph node', 
           'spleen': 'spleen', 'liver': 'liver', 'Pancreas': 'pancreas', 'Small Intestine': 'small intestine', 'large intestine': 'large intestine', 'kidney': 'kidney', 
           'urinary bladder': 'urinary bladder', 'ureter': 'ureter', 'prostate gland': 'prostate gland', 'ovary': 'ovary', 'Fallopian tube':'fallopian_tube', 
           'Uterus': 'uterus', 'Placenta': 'placenta', 'knee': 'knee', 'Bone marrow': 'bone marrow'}

plt.figure(frameon=False, figsize=(20,20))
plt.axes().set_aspect('equal')
plt.axis('off')
plt.xlim(0,1720)
plt.ylim(-1720,0)
for organ in coordinates_black.keys():
    size_scale = 40
    if mapping[organ] in match_vess.keys():
        red_diam = nomatch_vess[mapping[organ]]
        if red_diam:
            plt.scatter([coordinates_red[organ][0]], [-coordinates_red[organ][1]], s=red_diam*size_scale, c='tab:red', alpha=0.4)            
        else: 
            plt.scatter([coordinates_red[organ][0]], [-coordinates_red[organ][1]], s=1*size_scale, facecolors='none', edgecolors='tab:red', alpha=0.4) 
            
        black_diam = match_vess[mapping[organ]]
        if black_diam:
            plt.scatter([coordinates_black[organ][0]], [-coordinates_black[organ][1]], s=black_diam*size_scale, c='black', alpha=0.4)
        else:
            plt.scatter([coordinates_black[organ][0]], [-coordinates_black[organ][1]], s=1*size_scale,  facecolors='none', edgecolors='black', alpha=0.4)
        
    else:
        plt.scatter([coordinates_red[organ][0]], [-coordinates_red[organ][1]], s=1*size_scale, facecolors='none', edgecolors='tab:red', alpha=0.4)
        plt.scatter([coordinates_black[organ][0]], [-coordinates_black[organ][1]], s=1*size_scale,  facecolors='none', edgecolors='black', alpha=0.4)
    
plt.savefig(f'viz_v13/male_bubbles_v13.svg', bbox_inches='tight', pad_inches=0)
   
plt.show()

In [ ]:
with open('viz_v13/female_wing_viz_v13.json', encoding='utf8') as json_file:
    viz = json.load(json_file)

In [ ]:
female_wing_viz = viz['data'][0]['values']

In [ ]:
body_x = female_wing_viz[0]['x']
body_y = female_wing_viz[0]['y']
# print('Body original: ', body_x, body_y)
# center_x = 1720/2
# center_y = 1720/2
# print('Center:', center_x, center_y)
coordinates_black = dict()
coordinates_red = dict()
for asct in female_wing_viz:
    if asct['name'] in fem_nodes[fem_nodes['id_old']==1].name.values:
        # male_rots_for_organs[asct['name']] = -round(np.arctan((asct['x']-body_x)/(body_y-asct['y']))*180/np.pi, 3)
        coordinates_red[asct['name']] = (round(body_x-10 + 13.3*(asct['x']-body_x),2), round(body_y + 13.3*(asct['y']-body_y), 2))
        coordinates_black[asct['name']] = (round(body_x-10 + 12*(asct['x']-body_x),2), round(body_y + 12*(asct['y']-body_y), 2))

mapping = {'Respiratory System': 'lung', 'heart': 'heart', 'brain': 'brain', 'eye': 'eye', 'skin': 'skin', 'thoracic thymus': 'thymus', 'lymph node': 'lymph node', 
           'spleen': 'spleen', 'liver': 'liver', 'Pancreas': 'pancreas', 'Small Intestine': 'small intestine', 'large intestine': 'large intestine', 'kidney': 'kidney', 
           'urinary bladder': 'urinary bladder', 'ureter': 'ureter', 'prostate gland': 'prostate gland', 'ovary': 'ovary', 'Fallopian tube':'fallopian_tube', 
           'Uterus': 'uterus', 'Placenta': 'placenta', 'knee': 'knee', 'Bone marrow': 'bone marrow'}

plt.figure(frameon=False, figsize=(20,20))
plt.axes().set_aspect('equal')
plt.axis('off')
plt.xlim(0,1720)
plt.ylim(-1720,0)
for organ in coordinates_black.keys():
    size_scale = 40
    if mapping[organ] in match_vess.keys():
        red_diam = nomatch_vess[mapping[organ]]
        if red_diam:
            plt.scatter([coordinates_red[organ][0]], [-coordinates_red[organ][1]], s=red_diam*size_scale, c='tab:red', alpha=0.4)            
        else: 
            plt.scatter([coordinates_red[organ][0]], [-coordinates_red[organ][1]], s=1*size_scale, facecolors='none', edgecolors='tab:red', alpha=0.4) 
            
        black_diam = match_vess[mapping[organ]]
        if black_diam:
            plt.scatter([coordinates_black[organ][0]], [-coordinates_black[organ][1]], s=black_diam*size_scale, c='black', alpha=0.4)
        else:
            plt.scatter([coordinates_black[organ][0]], [-coordinates_black[organ][1]], s=1*size_scale,  facecolors='none', edgecolors='black', alpha=0.4)
        
    else:
        plt.scatter([coordinates_red[organ][0]], [-coordinates_red[organ][1]], s=1*size_scale, facecolors='none', edgecolors='tab:red', alpha=0.4)
        plt.scatter([coordinates_black[organ][0]], [-coordinates_black[organ][1]], s=1*size_scale,  facecolors='none', edgecolors='black', alpha=0.4)
    
plt.savefig(f'viz_v13/female_bubbles_v13.svg', bbox_inches='tight', pad_inches=0)
   
plt.show()

In [ ]:
body_y

### Size legend

In [ ]:
body_x = female_wing_viz[0]['x']
body_y = female_wing_viz[0]['y']
# print('Body original: ', body_x, body_y)
# center_x = 1720/2
# center_y = 1720/2
# print('Center:', center_x, center_y)
# sizes_black = {'one': 1, 'twntyfour': 24, 'fortyeight': 48, 'seventyone': 71}
# coordinates_black = {'one': (body_x, body_y-200), 'twntyfour': (body_x, body_y-100), 'fortyeight': (body_x, body_y), 'seventyone': (body_x, body_y+100)}
sizes_black = {'one': 1, '35': 35, 'seventyone': 71}
coordinates_black = {'one': (body_x, body_y-75), '35': (body_x, body_y), 'seventyone': (body_x, body_y+75)}

plt.figure(frameon=False, figsize=(20,20))
plt.axes().set_aspect('equal')
plt.axis('off')
plt.xlim(0,1720)
plt.ylim(-1720,0)
for organ in coordinates_black.keys():
    size_scale = 40

    plt.scatter([coordinates_black[organ][0]], [-coordinates_black[organ][1]], s=sizes_black[organ]*size_scale,alpha=0.5, c='tab:grey',)# facecolors='none', edgecolors='tab:grey')
        
    
plt.savefig(f'viz_v13/bubblesizelegend.svg', bbox_inches='tight', pad_inches=0)
   
plt.show()

# Vascular network 

In [ ]:
matching_nodes = vasc_data[(vasc_data['is_in_organ_table']==1)&(vasc_data.BodyPart!='testis')]

In [ ]:
vasc_edges = vasc_data[['BranchesFrom', 'Vessel', 'VesselType']].copy()

In [ ]:
# vasc_edges = pd.read_excel('./Blood_Vasculature_v1.3_EXTENDED.xlsx', sheet_name='Vessel')[['BranchesFrom', 'Vessel', 'VesselType']]
vasc_edges

In [ ]:
matching_nodes

In [ ]:
vasc_edges = vasc_edges.rename(columns={'BranchesFrom':'source', 'Vessel': 'target'})
# vasc_edges.to_excel('vasc_edgelist.xlsx')

In [ ]:
#Removal of the self-loops
vasc_edges = vasc_edges[vasc_edges.source!=vasc_edges.target]

In [ ]:
vasc_graph = nx.from_pandas_edgelist(vasc_edges)

In [ ]:
layout_pos_full = nx.drawing.nx_agraph.graphviz_layout(vasc_graph)

In [ ]:
plt.figure(figsize=(10,10))
nx.draw(vasc_graph, pos=layout_pos_full, node_size=10)
nx.draw_networkx_nodes(vasc_graph, layout_pos_full, nodelist=['right ventricle'], node_size=10, node_color='tab:orange')
plt.show()

### Construction of the pruned graph

The graph has to be pruned at the matching nodes

We start with the layer of matching nodes, and then in each iteration, we get the parent nodes (vessels), i.e. we are moving towards the core of the network in layers. 

In [ ]:
layer = vasc_edges[vasc_edges.target.isin(matching_nodes.Vessel)]
layers = [layer]
indices = list(layer.index)
while len(layer):
    layer = vasc_edges[(vasc_edges.target.isin(layer.source)) & (~vasc_edges.index.isin(indices))]
    layers += [layer]
    indices += list(layer.index)

In [ ]:
pruned_vasc_edges = vasc_edges[vasc_edges.index.isin(indices)]

In [ ]:
len(pruned_vasc_edges)

Nodes of the pruned network:

In [ ]:
vasc_data[vasc_data['Vessel'].isin(set(pruned_vasc_edges[['source', 'target']].values.flatten()))]

In [ ]:
vasc_data[vasc_data['Vessel'].isin(set(pruned_vasc_edges[['source', 'target']].values.flatten()))][['BodyPart', 'Vessel', 'ASID', 'VesselType']].to_excel('./viz_v14/nodes_of_vascular_network_v14.xlsx')

In [ ]:
pruned_vasc_edges.to_excel('viz_v14/vasc_edgelist_v14.xlsx')
vasc_edges.to_excel('viz_v14/vasc_edgelist_all_v14.xlsx')

In [ ]:
pruned_vasc_graph = nx.from_pandas_edgelist(pruned_vasc_edges)

In [ ]:
len(pruned_vasc_graph)

In [ ]:
pruned_vasc_graph = nx.induced_subgraph(vasc_graph, set(pruned_vasc_graph.nodes).union(set(matching_nodes.Vessel.values)))

In [ ]:
len(pruned_vasc_graph)

Note that the network is not connected, hence we have to add manually the 'heart' node and connect it to the components

In [ ]:
layout_pos = nx.drawing.nx_agraph.graphviz_layout(pruned_vasc_graph)
nx.draw(pruned_vasc_graph, pos=layout_pos, node_size=10)

### Get the coordinates from the Vega visualization 

In [ ]:
with open('viz_v14/female_wing_viz_v14.json', encoding='utf8') as json_file:
    viz = json.load(json_file)

In [ ]:
female_viz = viz['data'][0]['values']

Merge of heart & heart chamber, thyroid gland & lung and rename pelvis to placenta

In [ ]:
matching_nodes['BodyPart'].value_counts()

In [ ]:
organ_mapping = {'eye': 'eye', 'heart': 'heart', 
                 'heart chamber': 'heart', # heart chamber -> heart
                 'kidney': 'kidney',  'liver': 'liver', 'lung': 'lung', 'ovary': 'ovary', 
                 'pelvis': 'placenta_full_term', #pelvis -> placenta
                 'spleen': 'spleen', 'thymus': 'thymus', 
                 'thyroid gland': 'lung', #thyroid gland -> lung
                 'uterus': 'uterus'}

In [ ]:
matching_nodes['organ'] = matching_nodes['BodyPart'].apply(lambda x: organ_mapping[x])

Get the coordinates for the vessels

## Full graph

In [ ]:
matching_nodes['ASID'].apply(transform_id).values

In [ ]:
def get_coordinates_4_vessels(visualization, graph=pruned_vasc_graph):
    name = []
    name_in_vasc = []
    coords=[]
    organs = []
    for node in visualization:
        if transform_id(node['ontology_id']) in matching_nodes['ASID'].apply(transform_id).values: #if the node is in the vascular data 
            # if node['organ']==matching_nodes[matching_nodes['ASID'].apply(transform_id)==transform_id(node['ontology_id'])]['organ'].values[0]: # and if the organs match
            name += [node['name']]
            name_in_vasc += [matching_nodes[matching_nodes['ASID'].apply(transform_id)==transform_id(node['ontology_id'])]['Vessel'].values[0]]
            coords += [np.array([node['x'], -node['y']])]
            organs += [node['organ']]

    for node in graph:
        if node not in name_in_vasc:
            name += [node]
            name_in_vasc += [node]
            coords += [np.nan]
            organs += [np.nan]

    art_data = pd.DataFrame.from_dict({'name': name, 'name_in_vasc': name_in_vasc,
                                       'organ': organs, 'coords': coords})
    
    art_data = art_data.drop_duplicates(subset=['name_in_vasc', 'organ']).reset_index(drop=True) 
    
    art_data = art_data[art_data.name_in_vasc.isin(graph.nodes())]
    
    return art_data

In [ ]:
matching_nodes.to_excel('matching_nodes_v14.xlsx')

In [ ]:
coords = get_coordinates_4_vessels(female_viz)

In [ ]:
coords

In [ ]:
len(matching_nodes)

In [ ]:
len(coords.dropna())

In [ ]:
nodes_wo_coords = [node for node in coords[coords['coords'].isna()]['name_in_vasc'].values if node in matching_nodes['Vessel'].values ]
nodes_wo_coords

In [ ]:
matching_nodes[matching_nodes['Vessel'].isin(nodes_wo_coords)]

In [ ]:
nodes[nodes['ontology_id'].apply(transform_id).isin(matching_nodes[matching_nodes['Vessel'].isin(nodes_wo_coords)]['ASID'].apply(transform_id).values)]

In [ ]:
relabel_mapping = coords['name_in_vasc'].to_dict()
relabel_mapping = {v: k for k, v in relabel_mapping.items()}

In [ ]:
renamed_pruned_vasc = nx.relabel_nodes(pruned_vasc_graph, relabel_mapping)

In [ ]:
pos = coords['coords'].dropna().to_dict()

In [ ]:
pos2 = nx.spring_layout(renamed_pruned_vasc, pos=pos, fixed=list(pos.keys()), iterations=1000)

In [ ]:
nx.draw(renamed_pruned_vasc, pos2, node_size=10)
nx.draw_networkx_nodes(renamed_pruned_vasc, pos2, nodelist=list(pos.keys()), node_color='tab:orange', node_size=10)
# plt.savefig('small_network.png', dpi=300)
plt.show()

In [ ]:
connected_comps = [pruned_vasc_graph.subgraph(c).copy() for c in nx.connected_components(pruned_vasc_graph)]

In [ ]:
for comp in connected_comps:
    nx.draw(comp, node_size=10)
    plt.show()

In [ ]:
nx.draw(pruned_vasc_graph, pos=layout_pos, node_size=10)
plt.show()

In [ ]:
leaves_of_pruned = [node for node in pruned_vasc_graph if nx.degree(pruned_vasc_graph, node)<=1]

In [ ]:
len(leaves_of_pruned)

In [ ]:
matching_not_leaf = [node for node in matching_nodes.Vessel.values if node not in leaves_of_pruned]

In [ ]:
matching_leaf = [node for node in matching_nodes.Vessel.values if node in leaves_of_pruned]

In [ ]:
len(matching_not_leaf)+len(matching_leaf)

In [ ]:
len(matching_nodes)

In [ ]:
nx.draw(pruned_vasc_graph, pos=layout_pos, node_size=10)
nx.draw_networkx_nodes(pruned_vasc_graph, layout_pos,
                       nodelist=matching_not_leaf, 
                       node_color='tab:red', node_size=12)
nx.draw_networkx_nodes(pruned_vasc_graph, layout_pos,
                       nodelist=matching_leaf, 
                       node_color='tab:orange', node_size=12)
plt.show()

In [ ]:
plt.figure(figsize=(20,20))
nx.draw_networkx_nodes(vasc_graph, pos=layout_pos_full, node_size=5, node_color='tab:grey', alpha=0.3)
nx.draw_networkx_edges(vasc_graph, pos=layout_pos_full, alpha=0.1)
nx.draw_networkx_edges(pruned_vasc_graph, pos=layout_pos_full, alpha=0.5, edge_color='tab:red')
nx.draw_networkx_nodes(pruned_vasc_graph, layout_pos_full,
                       nodelist=pruned_vasc_graph.nodes, 
                       node_color='tab:red', node_size=12)
nx.draw_networkx_nodes(pruned_vasc_graph, layout_pos_full,
                       nodelist=matching_not_leaf, 
                       node_color='tab:orange', node_size=12)
nx.draw_networkx_nodes(pruned_vasc_graph, layout_pos_full,
                       nodelist=matching_leaf, 
                       node_color='tab:orange', node_size=12)
plt.tight_layout()
plt.savefig('vascular_v14.png', dpi=300)
plt.show()

# Connect the components of the vascular network 

In [ ]:
# Load json files 
json_data_asctb_blood = {}
for filename in listdir('./asct+b/v14/'):
    if '.json' in filename:
        # print(filename)
        with open(f'./asct+b/v14/{filename}', encoding='utf8') as json_file:
            data = json.load(json_file)

        json_data_asctb_blood[filename[:-5]] = data

In [ ]:
json_data_asctb_blood.keys()

In [ ]:
nodes, edges = combine_data(json_data_asctb_blood, remove_ctct=True, combine=False, blood=True)

In [ ]:
blood_vasc_nodes = nodes['blood_vasculature']

In [ ]:
blood_vasc_edges = edges['blood_vasculature']

In [ ]:
blood_vasc_nodes

In [ ]:
blood_vasc_graph = nx.from_pandas_edgelist(blood_vasc_edges)

In [ ]:
blood_vasc_graph = nx.relabel_nodes(blood_vasc_graph, blood_vasc_nodes['name'].to_dict())

In [ ]:
pos = nx.drawing.nx_pydot.graphviz_layout(G=blood_vasc_graph, root='blood vasculature')

In [ ]:
[node for node in pruned_vasc_graph if node not in blood_vasc_graph]

In [ ]:
subgraph = nx.induced_subgraph(blood_vasc_graph, list(pruned_vasc_graph.nodes)+['blood vasculature'])

In [ ]:
pos = nx.drawing.nx_pydot.graphviz_layout(subgraph, root='blood vasculature')

In [ ]:
pos_of_pruned = nx.spring_layout(pruned_vasc_graph, pos=pos, fixed=pos.keys(), iterations=1000, dim=2, k=0.01/np.sqrt(len(pruned_vasc_graph)))

In [ ]:
connected_comps = [pruned_vasc_graph.subgraph(c).copy() for c in nx.connected_components(pruned_vasc_graph)]

In [ ]:
new_edges = [edge for edge in subgraph.edges if edge not in pruned_vasc_graph.edges]

In [ ]:
connected_comps[3].nodes

In [ ]:
plt.figure(figsize=(20,20))
nx.draw_networkx_nodes(subgraph, pos=pos, node_size=5, node_color='tab:grey', alpha=0.3)
nx.draw_networkx_labels(subgraph, pos=pos, font_size=8)
nx.draw_networkx_edges(subgraph, pos=pos, edgelist=new_edges, alpha=0.9)

colors = ['tab:red', 'tab:blue', 'tab:blue', 'tab:red']
for idx, comp in enumerate(connected_comps):
    nx.draw_networkx_edges(comp, pos=pos_of_pruned, alpha=0.3, edge_color=colors[idx])

plt.tight_layout()
plt.savefig('completed_vascular_v14.png', dpi=350)
plt.show()

In [ ]:
new_edges

In [ ]:
[edge for edge in pruned_vasc_graph.edges if edge[0] in ['hepatic portal vein']]

### add edges

In [ ]:
pruned_vasc_graph_comp = pruned_vasc_graph.copy()

In [ ]:
pruned_vasc_graph_comp.add_edges_from([('blood vasculature', 'left atrium'),('blood vasculature', 'left ventricle'),
                                       ('blood vasculature', 'right atrium'), ('blood vasculature', 'right ventricle')])

In [ ]:
connected_comps[2].nodes

In [ ]:
veins = [node for node in pruned_vasc_graph_comp if (node not in connected_comps[0]) and (node not in connected_comps[3])]

In [ ]:
veins

In [ ]:
arteries = [node for node in pruned_vasc_graph_comp if (node not in connected_comps[1]) and (node not in connected_comps[2])]

In [ ]:
arteries

In [ ]:
pos = nx.drawing.nx_pydot.graphviz_layout(pruned_vasc_graph_comp, root='blood vasculature')

In [ ]:
plt.figure(figsize=(20,20))
nx.draw_networkx_nodes(pruned_vasc_graph_comp, pos=pos, node_size=5, node_color='tab:grey', alpha=0.3)
nx.draw_networkx_labels(pruned_vasc_graph_comp, pos=pos, font_size=8)
# nx.draw_networkx_edges(subgraph, pos=pos, edgelist=new_edges, alpha=0.9)

nx.draw_networkx_edges(nx.induced_subgraph(pruned_vasc_graph_comp, veins), pos=pos, alpha=0.5, edge_color='tab:blue')
nx.draw_networkx_edges(nx.induced_subgraph(pruned_vasc_graph_comp, arteries), pos=pos, alpha=0.5, edge_color='tab:red')

plt.tight_layout()
plt.savefig('completed_vascular_v14.png', dpi=350)
plt.show()

# Female and male vascular network 

In [ ]:
vasc_data.BodyPart.unique()

In [ ]:
female_vessels = vasc_data[vasc_data.BodyPart.isin(['clitoris', 'ovary', 'uterus', 'vagina'])] 

In [ ]:
male_vessels = vasc_data[vasc_data.BodyPart.isin(['penis', 'scrotum', 'testis'])]

In [ ]:
pruned_vasc_graph_comp_male = nx.induced_subgraph(pruned_vasc_graph_comp, [node for node in pruned_vasc_graph_comp if node not in female_vessels.Vessel.values])

In [ ]:
pruned_vasc_graph_comp_female = nx.induced_subgraph(pruned_vasc_graph_comp, [node for node in pruned_vasc_graph_comp if node not in male_vessels.Vessel.values])

In [ ]:
nx.write_gml(pruned_vasc_graph_comp_female, './viz_v14/pruned_vasc_graph_comp_female.gml')

In [ ]:
nx.write_gml(pruned_vasc_graph_comp_male, './viz_v14/pruned_vasc_graph_comp_male.gml')

In [ ]:
readojfwfoj=nx.read_gml('./viz_v14/pruned_vasc_graph_comp_male.gml')

In [ ]:
set(list(readojfwfoj.edges()))==set(list(pruned_vasc_graph_comp_male.edges()))

In [ ]:
pruned_vasc_graph_comp_female.number_of_edges()

In [ ]:
len(pruned_vasc_graph_comp_male)

In [ ]:
len(pruned_vasc_graph_comp_female)

In [ ]:
len(pruned_vasc_graph_comp)

## Unfortunately, this does not work well for the matplotlib inline backend in Jupyter because that backend uses a different default of rcParams['figure.dpi'] = 72. Additionally, it saves the figure with bbox_inches='tight', which crops the figure and makes the actual size unpredictable.

https://matplotlib.org/stable/gallery/subplots_axes_and_figures/figure_size_units.html

# So it is not recommended to run in Jupyter but it works fine in PyCharm / DataSpell

In [ ]:
def plot_fixed_graph2(graph, viz, bundle_edges=False, draw_labels=False, filename='vasculature'):
    graphs = {'veins': nx.induced_subgraph(graph, veins), 'arteries': nx.induced_subgraph(graph, arteries)}
    coords = {'veins': get_coordinates_4_vessels(viz, graphs['veins']), 'arteries': get_coordinates_4_vessels(viz, graphs['arteries'])}
    relabel_mapping_inv = {'veins': coords['veins']['name_in_vasc'].to_dict(), 'arteries': coords['arteries']['name_in_vasc'].to_dict()}
    relabel_mapping =  {'veins': {v: k for k, v in relabel_mapping_inv['veins'].items()}, 'arteries':{v: k for k, v in relabel_mapping_inv['arteries'].items()}}
    # {'veins': , 'arteries': }
    renamed_comp = {'veins': nx.relabel_nodes(graphs['veins'], relabel_mapping['veins']), 'arteries': nx.relabel_nodes(graphs['arteries'], relabel_mapping['arteries']) } 
    
    pos = {'veins': coords['veins']['coords'].dropna().to_dict(), 'arteries': coords['arteries']['coords'].dropna().to_dict()}
    pos['veins'][relabel_mapping['veins']['blood vasculature']] = np.array([body_x-10, -body_y])
    pos['arteries'][relabel_mapping['arteries']['blood vasculature']] = np.array([body_x-10, -body_y])
    pos2 = {'veins': nx.spring_layout(renamed_comp['veins'], pos=pos['veins'], fixed=list(pos['veins'].keys()), seed=10, iterations=1000, dim=2, k=0.01/np.sqrt(len(graphs['veins']))), 
            'arteries': nx.spring_layout(renamed_comp['arteries'], pos=pos['arteries'], fixed=list(pos['arteries'].keys()), seed=10, iterations=1000, dim=2, k=0.01/np.sqrt(len(graphs['arteries'])))}
    
    if not bundle_edges:
        plt.figure(figsize=(17.2,17.2))
        plt.axes().set_aspect('equal')
        plt.margins(x=0, y=0)
        plt.xlim(0,1720)
        plt.ylim(-1720, 0)
        plt.subplots_adjust(left=0, right=1, top=1, bottom=0, hspace=0, wspace=0)
        plt.tight_layout(pad=0, h_pad=0, w_pad=0, rect=(0,0,1,1))
        plt.axis('off') 
        nx.draw(renamed_comp['veins'], pos2['veins'], node_size=10, edge_color='tab:blue', node_color='tab:blue')
        nx.draw(renamed_comp['arteries'], pos2['arteries'], node_size=10, edge_color='tab:red', node_color='tab:red')
        if draw_labels:
            nx.draw_networkx_labels(renamed_comp['veins'], pos2['veins'], labels=relabel_mapping_inv['veins'], font_size=3)
            nx.draw_networkx_labels(renamed_comp['arteries'], pos2['arteries'], labels=relabel_mapping_inv['arteries'], font_size=3)
        # nx.draw_networkx_nodes(renamed_comp, pos2, nodelist=list(pos.keys()), node_color='tab:orange', node_size=10)
        plt.subplots_adjust(left=0, right=1, top=1, bottom=0)
        plt.tight_layout(pad=0, h_pad=0, w_pad=0, rect=(0,0,1,1))
        plt.axis('off')
        plt.gca().set_axis_off()
        plt.subplots_adjust(top = 1, bottom = 0, right = 1, left = 0, hspace = 0, wspace = 0)
        plt.margins(0,0)
        plt.gca().xaxis.set_major_locator(plt.NullLocator())
        plt.gca().yaxis.set_major_locator(plt.NullLocator())
        plt.savefig(f'./viz_v14/{filename}_v14.pdf',  transparent=True, pad_inches=0.0, bbox_inches=0)
        plt.show()
    else:
        nodes_only = {'veins': pd.DataFrame.from_dict(relabel_mapping_inv['veins'], orient='index').rename(columns={0:'name'}), 
                      'arteries': pd.DataFrame.from_dict(relabel_mapping_inv['arteries'], orient='index').rename(columns={0:'name'})}
        
        nodes = {'veins': pd.DataFrame.from_dict(pos2['veins']).T.rename(columns={0:'x', 1:'y'}).join(nodes_only['veins'])[['name', 'x', 'y']], 
                 'arteries': pd.DataFrame.from_dict(pos2['arteries']).T.rename(columns={0:'x', 1:'y'}).join(nodes_only['arteries'])[['name', 'x', 'y']]}
        
        edges = {'veins': nx.to_pandas_edgelist(renamed_comp['veins'])[['source', 'target']], 
                 'arteries': nx.to_pandas_edgelist(renamed_comp['arteries'])[['source', 'target']]}
        # hb = hammer_bundle(nodes, edges, initial_bandwidth=0.03,tension=0.9, accuracy=8000)
        hb = {'veins': hammer_bundle(nodes['veins'], edges['veins'], initial_bandwidth=.015, decay=0.8, tension=0.99, accuracy=1000), 
              'arteries': hammer_bundle(nodes['arteries'], edges['arteries'], initial_bandwidth=.015, decay=0.8, tension=0.99, accuracy=1000)}
        
        plt.figure(figsize=(17.2, 17.2))
        plt.axes().set_aspect('equal', anchor="NW")
        plt.margins(x=0, y=0)
        plt.xlim(0,1720)
        plt.ylim(-1720, 0)
        plt.subplots_adjust(left=0, right=1, top=1, bottom=0, hspace=0, wspace=0)  
        plt.plot(hb['veins']['x'], hb['veins']['y'], color='tab:blue', alpha=0.8)
        plt.plot(hb['arteries']['x'], hb['arteries']['y'], color='tab:red', alpha=0.8)
        plt.subplots_adjust(left=0, right=1, top=1, bottom=0)
        plt.tight_layout(pad=0, h_pad=0, w_pad=0, rect=(0,0,1,1))
        plt.axis('off')
        plt.gca().set_axis_off()
        plt.subplots_adjust(top=1, bottom=0, right=1, left=0, hspace=0, wspace=0)
        plt.margins(0,0)
        plt.gca().xaxis.set_major_locator(plt.NullLocator())
        plt.gca().yaxis.set_major_locator(plt.NullLocator())
        plt.savefig(f'./viz_v14/{filename}_bundled_v14.pdf', pad_inches=0.0, transparent=True, bbox_inches=0)
        plt.show()

In [ ]:
dpi = plt.rcParams['figure.dpi']

In [ ]:
dpi

In [ ]:
with open('viz_v14/female_wing_viz_v14.json', encoding='utf8') as json_file:
    viz = json.load(json_file)

In [ ]:
female_wing_viz = viz['data'][0]['values']

In [ ]:
body_x = female_wing_viz[0]['x']
body_y = female_wing_viz[0]['y']

In [ ]:
plt.figure(figsize=(21,13))
nx.draw_networkx(pruned_vasc_graph_comp_female, pos=nx.nx_agraph.graphviz_layout(pruned_vasc_graph_comp_female), node_size=10, font_size=9)

In [ ]:
plt.figure(figsize=(21,13))
nx.draw_networkx(pruned_vasc_graph_comp_male, pos=nx.nx_agraph.graphviz_layout(pruned_vasc_graph_comp_male), node_size=10, font_size=9)

In [ ]:
plot_fixed_graph2(pruned_vasc_graph_comp_female, female_wing_viz, draw_labels=True, filename='female_wing')

In [ ]:
plot_fixed_graph2(pruned_vasc_graph_comp_female, female_wing_viz, bundle_edges=True, filename='female_wing')

In [ ]:
with open('viz_v14/male_wing_viz_v14.json', encoding='utf8') as json_file:
    viz = json.load(json_file)

In [ ]:
male_wing_viz = viz['data'][0]['values']

In [ ]:
plot_fixed_graph2(pruned_vasc_graph_comp_male, male_wing_viz, draw_labels=True, filename='male_wing')

In [ ]:
plot_fixed_graph2(pruned_vasc_graph_comp_male, male_wing_viz, bundle_edges=True, filename='male_wing')

# Adobe Illustrator

1. Canvas size: 1780 x 1780
2. Import female wing: 
    * X: 920
    * Y: 890
    * W: 1720
    * H: 1720
3. Import famele vascular network and do the layover
    * X: 920
    * Y: 890
    * W: 1720
    * H: 1720
4. Import the female bubbles 
5. Group the female wing and its components and mirror it 
6. Import male wing, vascular network, and bubbles